In [1]:
"""
Jupytext-style script for:
01_data_cleaning_and_merging.ipynb
"""



'\nJupytext-style script for:\n01_data_cleaning_and_merging.ipynb\n'

# Title and objectives

This notebook prepares cleaned and merged hydrological time series for subsequent LSTM modelling of salinity intrusion. It loads raw salinity, discharge, tidal level, and rainfall data from local folders, standardises them to a common time step, and applies simple physical range checks. Short gaps are interpolated for input variables (discharge, tide, rainfall), while salinity is left un-interpolated so that the target remains as observed. Finally, the notebook builds one merged dataset per salinity station, containing the salinity target and all input variables, and saves these to `data/clean/` for downstream modelling.



# Imports and basic configuration

This section imports the core libraries used for the workflow: `pandas` and `numpy` for data handling, and `pathlib` for file system paths. It also defines base directories, file paths for raw inputs, and configuration options such as the resampling time step. Physical ranges and aggregation rules are specified here so they can be changed in one place if needed. By default the notebook looks for raw data under `data/raw/`, but if that folder is missing and a `DATA/` folder is present (as in this repository), it will use `DATA/` instead. Discharge, tide, and rain input files are auto-detected from subfolders, but the dictionaries can be edited at the top of the notebook if you prefer to specify a curated subset.



In [2]:
import re
from pathlib import Path

import numpy as np
import pandas as pd

# Base directories
BASE_DIR = Path.cwd().resolve()

# Primary assumption: data/raw/ contains raw CSVs
RAW_DIR = BASE_DIR / "data" / "raw"

# Fallback for this repository: DATA/ contains raw CSVs
alt_raw_dir = BASE_DIR / "DATA"
if not RAW_DIR.exists() and alt_raw_dir.exists():
    RAW_DIR = alt_raw_dir

# Cleaned outputs go to data/clean/
CLEAN_DIR = BASE_DIR / "data" / "clean"
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

# Time step for resampling: "h" (hourly) or "D" (daily)
TIME_STEP = "h"  # change to "D" for daily

# Salinity configuration
SALINITY_FILE = RAW_DIR / "salinity_all_stations.csv"
SALINITY_TIME_COL = "datetime"


def infer_station_id_from_filename(filename: str) -> str:
    """
    Infer a station identifier from a file name.

    Rules:
    - If the stem contains [Station Name], return the bracketed part with spaces removed.
    - Else if the stem matches NAME_YYYY_YYYY, return NAME.
    - Otherwise return the stem.
    """
    stem = Path(filename).stem

    m_bracket = re.search(r"\[([^\]]+)\]", stem)
    if m_bracket:
        return m_bracket.group(1).replace(" ", "")

    m_range = re.match(r"^(.*)_\d{4}_\d{4}$", stem)
    if m_range:
        return m_range.group(1)

    return stem


# Auto-detect raw input files for this repository layout
SALINITY_DIR = RAW_DIR / "Hourly_Salinity_Time_Series_44stations"
Q_DIR = RAW_DIR / "Q"
WL_DIR = RAW_DIR / "WL"
PRE_DIR = RAW_DIR / "Pre"

# Dictionaries for discharge, tide (water levels), and rainfall inputs
DISCHARGE_FILES = {}
if Q_DIR.exists():
    for path in sorted(Q_DIR.glob("*.csv")):
        sid = infer_station_id_from_filename(path.name)
        key = f"Q_{sid}"
        DISCHARGE_FILES[key] = path

TIDE_FILES = {}
if WL_DIR.exists():
    for path in sorted(WL_DIR.glob("*.csv")):
        sid = infer_station_id_from_filename(path.name)
        key = f"H_{sid}"
        TIDE_FILES[key] = path

RAIN_FILES = {}
if PRE_DIR.exists():
    for path in sorted(PRE_DIR.glob("*.csv")):
        sid = infer_station_id_from_filename(path.name)
        key = f"rain_{sid}"
        RAIN_FILES[key] = path

# Aggregation rules when resampling
AGG_MAP = {
    "salinity": "mean",     # typical for concentration
    "discharge": "mean",    # mean discharge over interval
    "tide": "mean",         # mean water level over interval
    "rain": "sum",          # sum rainfall over interval
}

# Physical ranges for basic QC (min, max)
PHYSICAL_RANGES = {
    "salinity": (0.0, 60.0),      # g/L or ppt
    "discharge": (0.0, 50000.0),  # m^3/s
    "tide": (-5.0, 5.0),          # m relative to datum
    "rain": (0.0, 500.0),         # mm per interval
}

# Will be inferred after loading salinity data
SALINITY_STATION_COLS = None


def prepare_timeseries(df: pd.DataFrame, time_col: str | None, time_step: str, agg: str = "mean") -> pd.DataFrame:
    """
    Convert a time column to a datetime index, sort, and resample.

    Parameters
    ----------
    df : DataFrame
        Input data frame containing a time column or a DatetimeIndex.
    time_col : str or None
        Name of the time column if present. If ``None`` or ``\"auto\"``,
        the function will try to infer a suitable datetime column by
        looking for names containing 'timestamp', 'time', or 'date'.
    time_step : str
        Pandas offset alias, e.g. "H" or "D".
    agg : str
        Aggregation function name for resampling ("mean", "sum", etc.).

    Returns
    -------
    DataFrame with a DatetimeIndex resampled to the requested time step.
    """
    df = df.copy()

    col = time_col

    if col is None or str(col).lower() == "auto":
        # Try explicit salinity time column first if present
        if SALINITY_TIME_COL in df.columns:
            col = SALINITY_TIME_COL
        else:
            # Look for a column that looks like a timestamp
            candidates = []
            for c in df.columns:
                cl = str(c).lower()
                if "timestamp" in cl or "date" in cl or cl.startswith("time"):
                    candidates.append(c)
            if candidates:
                col = candidates[0]

    if col is not None and col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")
        df = df.dropna(subset=[col])
        df = df.set_index(col)
    elif isinstance(df.index, pd.DatetimeIndex):
        # Already indexed by time; no action required
        pass
    else:
        raise KeyError(
            f"Could not find a suitable time column in {list(df.columns)} "
            "and index is not a DatetimeIndex."
        )

    # Normalise to timezone-naive timestamps to allow joins across series.
    if isinstance(df.index, pd.DatetimeIndex) and df.index.tz is not None:
        df.index = df.index.tz_convert(None)

    df = df.sort_index()

    if time_step is not None:
        # Keep only numeric value columns for aggregation; discard metadata.
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) == 0:
            # No numeric columns to aggregate; return an empty frame with a DatetimeIndex.
            empty = df.iloc[0:0].copy()
            empty.index = pd.DatetimeIndex(empty.index)
            return empty
        df_numeric = df[numeric_cols]
        df = df_numeric.resample(time_step).agg(agg)

    return df


def clip_and_nan(df: pd.DataFrame, col_type: str) -> pd.DataFrame:
    """
    Clip values outside plausible physical ranges to NaN.

    Parameters
    ----------
    df : DataFrame
        Input data frame with numeric columns to be checked.
    col_type : str
        One of {"salinity", "discharge", "tide", "rain"}.

    Returns
    -------
    DataFrame with out-of-range values replaced by NaN.
    """
    if col_type not in PHYSICAL_RANGES:
        raise ValueError(f"Unknown col_type '{col_type}'. Expected one of {list(PHYSICAL_RANGES.keys())}.")

    low, high = PHYSICAL_RANGES[col_type]
    df_clipped = df.copy()

    numeric_cols = df_clipped.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        series = df_clipped[col]
        mask = (series < low) | (series > high)
        if mask.any():
            df_clipped.loc[mask, col] = np.nan

    return df_clipped


def missing_summary(df: pd.DataFrame, label: str = "") -> pd.DataFrame:
    """
    Print and return a simple missingness summary per column.

    The summary includes total row count (n), number of missing values,
    and percentage missing for each column.
    """
    n = len(df)
    n_missing = df.isna().sum()
    pct_missing = (n_missing / n * 100) if n > 0 else np.nan

    summary = pd.DataFrame(
        {
            "n": n,
            "n_missing": n_missing,
            "pct_missing": pct_missing,
        }
    )

    print("-" * 60)
    if label:
        print(f"Missingness summary for {label}")
    else:
        print("Missingness summary")
    print(summary.round(2))

    return summary


def interpolate_short_gaps(df: pd.DataFrame, limit: int = 3) -> pd.DataFrame:
    """
    Interpolate short gaps in each numeric column using time-based interpolation.

    Parameters
    ----------
    df : DataFrame
        Input data frame with a DatetimeIndex.
    limit : int
        Maximum number of consecutive NaNs to interpolate in one gap.

    Returns
    -------
    DataFrame with short gaps interpolated; longer gaps remain as NaN.
    """
    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError("interpolate_short_gaps expects a DataFrame with a DatetimeIndex.")

    df_interp = df.copy()

    for col in df_interp.columns:
        series = df_interp[col]
        if not np.issubdtype(series.dtype, np.number):
            continue
        df_interp[col] = series.interpolate(
            method="time",
            limit=limit,
            limit_direction="both",
        )

    return df_interp




# Load and inspect raw datasets

In this section we load the raw salinity, discharge, tidal level, and rainfall datasets. Salinity is taken either from a pre-combined wide file (`salinity_all_stations.csv`) if it exists, or, if not, from individual station CSVs under `Hourly_Salinity_Time_Series_44stations` which are merged into a single wide table. Station columns are identified automatically as all non-time, non-metadata columns. Discharge (`Q`), tide or water level (`WL`), and rainfall (`Pre`) files are auto-detected into dictionaries so that each series is named consistently; the shapes and column names are printed so that any unexpected structure can be checked before proceeding.



In [3]:
# Load raw salinity data
if SALINITY_FILE.exists():
    print(f"Loading wide salinity file from {SALINITY_FILE}")
    salinity_raw = pd.read_csv(SALINITY_FILE)
else:
    salinity_dir = SALINITY_DIR
    if not salinity_dir.exists():
        raise FileNotFoundError(
            f"Could not find salinity file {SALINITY_FILE} or directory {salinity_dir}. "
            "Please update SALINITY_FILE or RAW_DIR."
        )

    print(f"Building wide salinity table from per-station files in {salinity_dir}")
    sal_frames = []
    for path in sorted(salinity_dir.glob("*.csv")):
        station_id = infer_station_id_from_filename(path.name)
        df_station = pd.read_csv(path)
        if SALINITY_TIME_COL not in df_station.columns:
            raise KeyError(f"Expected time column '{SALINITY_TIME_COL}' in {path.name}")

        value_cols = [c for c in df_station.columns if c != SALINITY_TIME_COL]
        if not value_cols:
            raise ValueError(f"No value columns found in {path.name}")

        numeric_candidates = [
            c for c in value_cols if np.issubdtype(df_station[c].dtype, np.number)
        ]
        val_col = numeric_candidates[0] if numeric_candidates else value_cols[0]

        df_station = df_station[[SALINITY_TIME_COL, val_col]].rename(
            columns={val_col: station_id}
        )
        sal_frames.append(df_station)

    if not sal_frames:
        raise RuntimeError(f"No CSV files found in {salinity_dir}")

    salinity_raw = sal_frames[0]
    for df_station in sal_frames[1:]:
        salinity_raw = salinity_raw.merge(df_station, on=SALINITY_TIME_COL, how="outer")

if SALINITY_TIME_COL not in salinity_raw.columns:
    raise KeyError(f"Expected time column '{SALINITY_TIME_COL}' in salinity data.")

# Infer station columns as all non-time, non-metadata columns
metadata_candidates = {
    SALINITY_TIME_COL.lower(),
    "station",
    "station_id",
    "station_name",
}

SALINITY_STATION_COLS = [
    col for col in salinity_raw.columns
    if col.lower() not in metadata_candidates
]

print("Detected salinity station columns:")
print(SALINITY_STATION_COLS)
print(f"\nRaw salinity shape: {salinity_raw.shape}")
print(salinity_raw.head())

# Load raw discharge data
discharge_raw = {}
if not DISCHARGE_FILES:
    print("\nNo discharge files detected. DISCHARGE_FILES is empty.")
else:
    for name, path in DISCHARGE_FILES.items():
        if not Path(path).exists():
            print(f"\n[warning] Discharge file not found for '{name}': {path}")
            continue
        df = pd.read_csv(path)
        discharge_raw[name] = df
        print(f"\nLoaded discharge series '{name}' from {path} with shape {df.shape}")
        print(df.head())

# Load raw tidal level data
tide_raw = {}
if not TIDE_FILES:
    print("\nNo tide/water-level files detected. TIDE_FILES is empty.")
else:
    for name, path in TIDE_FILES.items():
        if not Path(path).exists():
            print(f"\n[warning] Tide file not found for '{name}': {path}")
            continue
        df = pd.read_csv(path)
        tide_raw[name] = df
        print(f"\nLoaded tide series '{name}' from {path} with shape {df.shape}")
        print(df.head())

# Load raw rainfall data
rain_raw = {}
if not RAIN_FILES:
    print("\nNo rainfall files detected. RAIN_FILES is empty.")
else:
    for name, path in RAIN_FILES.items():
        if not Path(path).exists():
            print(f"\n[warning] Rain file not found for '{name}': {path}")
            continue
        df = pd.read_csv(path)
        rain_raw[name] = df
        print(f"\nLoaded rain series '{name}' from {path} with shape {df.shape}")
        print(df.head())




Building wide salinity table from per-station files in C:\Users\User\Desktop\Disertation\DATA\Hourly_Salinity_Time_Series_44stations
Detected salinity station columns:
['AnDinh', 'AnLacTay', 'AnNinh', 'AnThuan', 'BenLuc', 'BenTrai', 'BinhDai', 'CaMau', 'CauNoi', 'CauQuan', 'DaiNgai', 'DongTam', 'GanhHao', 'GiongTrom', 'GoQuao', 'HoaBinh', 'HungMy', 'HuongMy', 'KhanhThanhTan', 'LocThuan', 'LongPhu', 'LuynhQuynh', 'MyHoa', 'MyTho', 'PhuocLong', 'RachGia', 'SocTrang', 'SonDoc', 'SongDoc', 'TamNgan', 'TanAn', 'ThanhPhu', 'ThoiBinh', 'TPBacLieu', 'TraKha', 'TranDe', 'TraVinh', 'TuyenNhon', 'VamKenh', 'VungLiem', 'XeoRo', 'XuanKhanh']

Raw salinity shape: (87282, 43)
         datetime  AnDinh  AnLacTay  AnNinh  AnThuan  BenLuc  BenTrai  \
0  1/1/2021 17:00     0.1       NaN     NaN      NaN     NaN      NaN   
1  1/1/2021 19:00     0.2       NaN     NaN      NaN     NaN      NaN   
2  1/1/2021 21:00     0.1       NaN     NaN      NaN     NaN      NaN   
3   1/1/2021 5:00     0.1       NaN   

# Standardise datetime and resample to a common time step

Here we convert the datetime columns to proper `datetime64` types, set them as indexes, and resample each series to a common time step. The default resolution is hourly (`TIME_STEP = "H"`), but this can be changed to daily (`"D"`) or another appropriate interval. Salinity and most other variables are aggregated using the mean, while rainfall is aggregated using the sum to preserve totals. This step ensures that all series share a consistent temporal grid, which simplifies later merging and modelling.



In [4]:
# Resample salinity: keep only time and station columns
salinity_cols = [SALINITY_TIME_COL] + SALINITY_STATION_COLS
salinity_ts = prepare_timeseries(
    salinity_raw[salinity_cols],
    time_col=SALINITY_TIME_COL,
    time_step=TIME_STEP,
    agg=AGG_MAP["salinity"],
)
print(f"Salinity resampled to {TIME_STEP}: shape = {salinity_ts.shape}")

# Resample discharge series
discharge_ts = {}
for name, df_raw in discharge_raw.items():
    df_ts = prepare_timeseries(
        df_raw,
        time_col="auto",
        time_step=TIME_STEP,
        agg=AGG_MAP["discharge"],
    )
    discharge_ts[name] = df_ts
    print(f"Discharge '{name}' resampled to {TIME_STEP}: shape = {df_ts.shape}")

# Resample tidal level series
tide_ts = {}
for name, df_raw in tide_raw.items():
    df_ts = prepare_timeseries(
        df_raw,
        time_col="auto",
        time_step=TIME_STEP,
        agg=AGG_MAP["tide"],
    )
    tide_ts[name] = df_ts
    print(f"Tide '{name}' resampled to {TIME_STEP}: shape = {df_ts.shape}")

# Resample rainfall series
rain_ts = {}
for name, df_raw in rain_raw.items():
    df_ts = prepare_timeseries(
        df_raw,
        time_col="auto",
        time_step=TIME_STEP,
        agg=AGG_MAP["rain"],
    )
    rain_ts[name] = df_ts
    print(f"Rain '{name}' resampled to {TIME_STEP}: shape = {df_ts.shape}")




Salinity resampled to h: shape = (239493, 42)
Discharge 'Q_MyThuan' resampled to h: shape = (61321, 2)
Discharge 'Q_CanTho' resampled to h: shape = (61321, 2)
Discharge 'Q_VamNao' resampled to h: shape = (61321, 2)
Tide 'H_TanChau' resampled to h: shape = (391465, 2)
Tide 'H_MyThuan' resampled to h: shape = (555841, 2)
Tide 'H_MyTho' resampled to h: shape = (245425, 2)
Tide 'H_ChoMoi' resampled to h: shape = (287089, 2)
Tide 'H_CanTho' resampled to h: shape = (391465, 2)
Tide 'H_DaiNgai' resampled to h: shape = (245401, 2)
Tide 'H_ChoLach' resampled to h: shape = (227881, 2)
Tide 'H_VamNao' resampled to h: shape = (341017, 2)
Tide 'H_VamKenh' resampled to h: shape = (279673, 2)
Tide 'H_TanHiep' resampled to h: shape = (245425, 2)
Tide 'H_ViThanh' resampled to h: shape = (245401, 2)
Tide 'H_CaiLay' resampled to h: shape = (227905, 2)
Tide 'H_PhungHiep' resampled to h: shape = (48034, 2)
Rain 'rain_MyThuan' resampled to h: shape = (151849, 2)
Rain 'rain_CaMau' resampled to h: shape = (28

# Basic quality control (ranges, missingness)

This section applies simple physical range checks to each variable group to remove impossible or highly implausible values. Values outside the specified ranges (for example negative salinity or rainfall, or excessively large discharges) are replaced by `NaN` and treated as missing. After clipping, missingness is summarised per variable so that data quality can be inspected before any interpolation is applied. Salinity is cleaned but not interpolated, as it serves as the prediction target.



In [5]:
# Range-based QC for salinity (no interpolation here)
salinity_clean = clip_and_nan(salinity_ts, col_type="salinity")
_ = missing_summary(salinity_clean, label="salinity (after range checks, no interpolation)")

# Range-based QC for discharge
discharge_qc = {}
for name, df_ts in discharge_ts.items():
    df_qc = clip_and_nan(df_ts, col_type="discharge")
    discharge_qc[name] = df_qc

# Range-based QC for tidal level
tide_qc = {}
for name, df_ts in tide_ts.items():
    df_qc = clip_and_nan(df_ts, col_type="tide")
    tide_qc[name] = df_qc

# Range-based QC for rainfall
rain_qc = {}
for name, df_ts in rain_ts.items():
    df_qc = clip_and_nan(df_ts, col_type="rain")
    rain_qc[name] = df_qc




------------------------------------------------------------
Missingness summary for salinity (after range checks, no interpolation)
                    n  n_missing  pct_missing
AnDinh         239493     221601        92.53
AnLacTay       239493     239493       100.00
AnNinh         239493     239493       100.00
AnThuan        239493     239493       100.00
BenLuc         239493     239493       100.00
BenTrai        239493     239493       100.00
BinhDai        239493     239493       100.00
CaMau          239493     239493       100.00
CauNoi         239493     239493       100.00
CauQuan        239493     239493       100.00
DaiNgai        239493     239493       100.00
DongTam        239493     239493       100.00
GanhHao        239493     239493       100.00
GiongTrom      239493     239493       100.00
GoQuao         239493     239493       100.00
HoaBinh        239493     239493       100.00
HungMy         239493     239493       100.00
HuongMy        239493     239493       

# Interpolate short gaps for inputs (not salinity)

Short gaps in the input variables (discharge, tidal levels, rainfall) can create unnecessary breaks in the time series for modelling. In this section we interpolate only short gaps, defined here as up to three consecutive missing values, using time-based interpolation. Longer gaps remain as missing and will be handled later during model preparation. Salinity is **not** interpolated, so the target reflects observed values only. After interpolation, we print missingness summaries for each cleaned input series.



In [6]:
# Interpolate short gaps and rename columns for inputs

discharge_clean = {}
for name, df_qc in discharge_qc.items():
    df_clean = interpolate_short_gaps(df_qc, limit=3)
    # Rename columns to include the series name
    if df_clean.shape[1] == 1:
        df_clean.columns = [name]
    else:
        df_clean.columns = [f"{name}_{col}" for col in df_clean.columns]
    discharge_clean[name] = df_clean
    _ = missing_summary(df_clean, label=f"discharge '{name}' (clean, interpolated)")

tide_clean = {}
for name, df_qc in tide_qc.items():
    df_clean = interpolate_short_gaps(df_qc, limit=3)
    if df_clean.shape[1] == 1:
        df_clean.columns = [name]
    else:
        df_clean.columns = [f"{name}_{col}" for col in df_clean.columns]
    tide_clean[name] = df_clean
    _ = missing_summary(df_clean, label=f"tide '{name}' (clean, interpolated)")

rain_clean = {}
for name, df_qc in rain_qc.items():
    df_clean = interpolate_short_gaps(df_qc, limit=3)
    if df_clean.shape[1] == 1:
        df_clean.columns = [name]
    else:
        df_clean.columns = [f"{name}_{col}" for col in df_clean.columns]
    rain_clean[name] = df_clean
    _ = missing_summary(df_clean, label=f"rain '{name}' (clean, interpolated)")




------------------------------------------------------------
Missingness summary for discharge 'Q_MyThuan' (clean, interpolated)
                            n  n_missing  pct_missing
Q_MyThuan_Station Code  61321      45150        73.63
Q_MyThuan_Value         61321      45150        73.63
------------------------------------------------------------
Missingness summary for discharge 'Q_CanTho' (clean, interpolated)
                           n  n_missing  pct_missing
Q_CanTho_Station Code  61321      45640        74.43
Q_CanTho_Value         61321      45640        74.43
------------------------------------------------------------
Missingness summary for discharge 'Q_VamNao' (clean, interpolated)
                           n  n_missing  pct_missing
Q_VamNao_Station Code  61321      61321       100.00
Q_VamNao_Value         61321      44142        71.99
------------------------------------------------------------
Missingness summary for tide 'H_TanChau' (clean, interpolated)
           

# Merge variables and create per-station datasets

Now we construct a common input feature matrix by aligning all cleaned discharge, tide, and rainfall series on the salinity time index. The merged input frame contains one column per input variable, with rows corresponding to time steps. For each salinity station, we then build a per-station data frame containing a single `salinity` column (the target) and all input variables at that station's time steps. Rows where salinity is missing are dropped so that the resulting datasets are ready for supervised learning.



In [7]:
# Build common input feature matrix X_inputs on salinity index
common_index = salinity_clean.index
X_inputs = pd.DataFrame(index=common_index)

# Join discharge inputs
for name, df_clean in discharge_clean.items():
    X_inputs = X_inputs.join(df_clean, how="outer")

# Join tidal level inputs
for name, df_clean in tide_clean.items():
    X_inputs = X_inputs.join(df_clean, how="outer")

# Join rainfall inputs
for name, df_clean in rain_clean.items():
    X_inputs = X_inputs.join(df_clean, how="outer")

# Optionally drop rows where all input variables are NaN
X_inputs = X_inputs.dropna(how="all")

print(f"Input feature matrix shape after merging: {X_inputs.shape}")
print(X_inputs.head())

# Create per-station datasets
station_data = {}
station_summaries = []

for station in SALINITY_STATION_COLS:
    sal_series = salinity_clean[station]

    n_before = len(sal_series)
    pct_missing_before = sal_series.isna().mean() * 100 if n_before > 0 else np.nan

    # Base frame on salinity index, with salinity as target
    df_station = pd.DataFrame(index=salinity_clean.index)
    df_station["salinity"] = sal_series

    # Inner join with inputs so that only common timestamps are retained
    df_station = df_station.join(X_inputs, how="inner")

    # Drop rows where salinity is missing
    df_station = df_station.dropna(subset=["salinity"])

    station_data[station] = df_station

    n_after = len(df_station)
    input_cols = [c for c in df_station.columns if c != "salinity"]
    if n_after > 0 and input_cols:
        pct_missing_inputs_after = df_station[input_cols].isna().mean().mean() * 100
    else:
        pct_missing_inputs_after = np.nan

    station_summaries.append(
        {
            "station": station,
            "n_before": n_before,
            "pct_missing_salinity_before": pct_missing_before,
            "n_after": n_after,
            "pct_missing_inputs_after": pct_missing_inputs_after,
            "start_after": df_station.index.min(),
            "end_after": df_station.index.max(),
        }
    )

# Inspect one example station (if available)
if station_data:
    first_station = next(iter(station_data))
    print(f"\nExample merged dataset for station '{first_station}':")
    print(station_data[first_station].head())




Input feature matrix shape after merging: (911080, 66)
                     Q_MyThuan_Station Code  Q_MyThuan_Value  \
1920-01-01 00:00:00                     NaN              NaN   
1920-01-01 01:00:00                     NaN              NaN   
1920-01-01 02:00:00                     NaN              NaN   
1920-01-01 03:00:00                     NaN              NaN   
1920-01-01 04:00:00                     NaN              NaN   

                     Q_CanTho_Station Code  Q_CanTho_Value  \
1920-01-01 00:00:00                    NaN             NaN   
1920-01-01 01:00:00                    NaN             NaN   
1920-01-01 02:00:00                    NaN             NaN   
1920-01-01 03:00:00                    NaN             NaN   
1920-01-01 04:00:00                    NaN             NaN   

                     Q_VamNao_Station Code  Q_VamNao_Value  \
1920-01-01 00:00:00                    NaN             NaN   
1920-01-01 01:00:00                    NaN             NaN   
1

# Save outputs and print summary

Finally, each station-specific merged dataset is written to `data/clean/` as a separate CSV file named `station_<station_name>.csv`. The index is saved as a `datetime` column so that it can be read easily in later notebooks or scripts. A compact summary table is printed showing, for each station, the number of rows and percentage of missing data before cleaning (for salinity) and after cleaning (for the inputs), along with the date range of the final dataset. This provides a quick overview of data availability for subsequent LSTM modelling experiments.



In [ ]:
# Save per-station datasets to disk
for station, df_station in station_data.items():
    # Make station name safe for filenames
    safe_station = "".join(
        c if c.isalnum() or c in ("-", "_") else "_"
        for c in str(station)
    )
    out_path = CLEAN_DIR / f"station_{safe_station}.csv"
    df_station.to_csv(out_path, index_label="datetime")
    print(f"Saved cleaned dataset for station {station!r} to {out_path}")

# Build and print overall summary table
summary_df = pd.DataFrame(station_summaries)

if not summary_df.empty:
    # Format dates for readability
    summary_df["start_after"] = summary_df["start_after"].dt.strftime("%Y-%m-%d %H:%M:%S")
    summary_df["end_after"] = summary_df["end_after"].dt.strftime("%Y-%m-%d %H:%M:%S")
    summary_df = summary_df.set_index("station")

    print("\nOverview of per-station datasets (before/after cleaning):")
    print(summary_df[[
        "n_before",
        "pct_missing_salinity_before",
        "n_after",
        "pct_missing_inputs_after",
        "start_after",
        "end_after",
    ]].round(2))
else:
    print("No station datasets were created; please check the salinity inputs and configuration.")


Saved cleaned dataset for station 'AnDinh' to C:\Users\User\Desktop\Disertation\data\clean\station_AnDinh.csv
Saved cleaned dataset for station 'AnLacTay' to C:\Users\User\Desktop\Disertation\data\clean\station_AnLacTay.csv
Saved cleaned dataset for station 'AnNinh' to C:\Users\User\Desktop\Disertation\data\clean\station_AnNinh.csv
Saved cleaned dataset for station 'AnThuan' to C:\Users\User\Desktop\Disertation\data\clean\station_AnThuan.csv
Saved cleaned dataset for station 'BenLuc' to C:\Users\User\Desktop\Disertation\data\clean\station_BenLuc.csv
Saved cleaned dataset for station 'BenTrai' to C:\Users\User\Desktop\Disertation\data\clean\station_BenTrai.csv
Saved cleaned dataset for station 'BinhDai' to C:\Users\User\Desktop\Disertation\data\clean\station_BinhDai.csv
Saved cleaned dataset for station 'CaMau' to C:\Users\User\Desktop\Disertation\data\clean\station_CaMau.csv
Saved cleaned dataset for station 'CauNoi' to C:\Users\User\Desktop\Disertation\data\clean\station_CauNoi.csv
Sa

: 